In [132]:
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, jwt_required, get_jwt_identity
import mysql.connector
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from werkzeug.serving import run_simple
import mysql.connector



In [133]:
app = Flask(__name__)
app.config['JWT_SECRET_KEY'] = 'jwt-secret-key'  
jwt = JWTManager(app)


In [134]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'authentication'
}

try:
    conn = mysql.connector.connect(**db_config)
    if conn.is_connected():
        print("Connected to the database")
    conn.close()
except mysql.connector.Error as err:
    print(f"Error: {err}")

Connected to the database


In [135]:
def get_data_from_db(query, params=None):
    try:
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor(dictionary=True)
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        data = cursor.fetchall()
        cursor.close()
        conn.close()
        return pd.DataFrame(data)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return pd.DataFrame()

In [136]:
@app.route('/fetch_volunteers', methods=['GET'])
def fetch_volunteers():
    query = "SELECT id, name, skills, availability FROM users"
    volunteers = get_data_from_db(query)
    return volunteers.to_json(orient='records'), 200

In [137]:
@app.route('/fetch_opportunities', methods=['GET'])
def fetch_opportunities():
    query = "SELECT id, title, skills, availability FROM opportunities"
    opportunities = get_data_from_db(query)
    return opportunities.to_json(orient='records'), 200

In [138]:
def calculate_availability_match(vol_avail, opp_avail):
    if vol_avail == 'flexible' or opp_avail == 'flexible':
        return 1.0
    elif vol_avail == opp_avail:
        return 1.0
    else:
        return 0.0

In [139]:
def match_opportunities(volunteer, opportunities):
    # Prepare TF-IDF vectorizer
    tfidf = TfidfVectorizer()
    
    # Combine volunteer skills into a single string
    volunteer_skills = ' '.join(volunteer['skills'].split(','))
    
    # Prepare opportunity skills
    opportunities['all_skills'] = opportunities['skills'].apply(lambda x: ' '.join(x.split(',')))
    
    # Fit and transform the TF-IDF vectorizer
    all_skills = [volunteer_skills] + opportunities['all_skills'].tolist()
    tfidf_matrix = tfidf.fit_transform(all_skills)
    
    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    
    # Calculate availability match
    availability_matches = opportunities['availability'].apply(
        lambda x: calculate_availability_match(volunteer['availability'], x)
    )
    
    # Combine similarity scores and availability matches
    opportunities['skill_similarity'] = cosine_similarities
    opportunities['availability_match'] = availability_matches
    opportunities['total_score'] = (opportunities['skill_similarity'] + opportunities['availability_match']) / 2
    
    # Sort opportunities by total score in descending order
    sorted_opportunities = opportunities.sort_values('total_score', ascending=False)
    
    # Classify opportunities as matched or unmatched
    threshold = 0.5  # Adjust this threshold as needed
    sorted_opportunities['matched'] = sorted_opportunities['total_score'] > threshold
    
    return sorted_opportunities

In [140]:
@app.route('/recommendations', methods=['GET'])
@jwt_required()
def get_recommendations():
    current_user_id = get_jwt_identity()
    
    # Fetch volunteer data
    volunteer_query = "SELECT id, name, skills, availability FROM users WHERE id = %s"
    volunteer = get_data_from_db(volunteer_query, (current_user_id,)).iloc[0]
    
    # Fetch all opportunities
    opportunities_query = "SELECT id, title, skills, availability FROM opportunities"
    opportunities = get_data_from_db(opportunities_query)
    
    if opportunities.empty:
        return jsonify({"error": "No opportunities found"}), 404
    
    # Match opportunities
    matched_opportunities = match_opportunities(volunteer, opportunities)
    
    # Prepare the response
    response = {
        "matched_opportunities": matched_opportunities[matched_opportunities['matched']].to_dict(orient='records'),
        "unmatched_opportunities": matched_opportunities[~matched_opportunities['matched']].to_dict(orient='records')
    }
    
    return jsonify(response), 200


In [141]:
if __name__ == "__main__":
    run_simple('localhost', 5001, app, use_reloader=False, use_debugger=True)

 * Running on http://localhost:5001
Press CTRL+C to quit
